In [1]:
import re
import sys
import json


import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from datatools.analyzer import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules import loss
import torch.optim as optim
import torch.nn.utils.rnn as rnn

In [71]:
def pooling_1(A):
    b_len = len(A)
    f_len = len(A[0][0])
    pooled = torch.zeros((b_len, f_len)).cuda()
    for i, batch in enumerate( A ):
        for j in range(f_len):
                # batch_pooled.append( A[i, torch.argmax(A[i, :, j]), j] )
            pooled[i, j] = A[i, torch.argmax(A[i, :, j]), j]
    return pooled

In [72]:
def pooling_2(A):
    # A : dim3
    if len(A.shape) == 2:
        A = torch.stack([A])
    b_len = len(A)
    seq_len = len(A[0])
    m = nn.MaxPool1d(seq_len, stride=seq_len)
    B = A.permute((0, 2, 1))
    return m(B).reshape(b_len, -1)

In [73]:
A =  torch.arange(6000).reshape(5, 2, -1).float()
A

tensor([[[0.0000e+00, 1.0000e+00, 2.0000e+00,  ..., 5.9700e+02,
          5.9800e+02, 5.9900e+02],
         [6.0000e+02, 6.0100e+02, 6.0200e+02,  ..., 1.1970e+03,
          1.1980e+03, 1.1990e+03]],

        [[1.2000e+03, 1.2010e+03, 1.2020e+03,  ..., 1.7970e+03,
          1.7980e+03, 1.7990e+03],
         [1.8000e+03, 1.8010e+03, 1.8020e+03,  ..., 2.3970e+03,
          2.3980e+03, 2.3990e+03]],

        [[2.4000e+03, 2.4010e+03, 2.4020e+03,  ..., 2.9970e+03,
          2.9980e+03, 2.9990e+03],
         [3.0000e+03, 3.0010e+03, 3.0020e+03,  ..., 3.5970e+03,
          3.5980e+03, 3.5990e+03]],

        [[3.6000e+03, 3.6010e+03, 3.6020e+03,  ..., 4.1970e+03,
          4.1980e+03, 4.1990e+03],
         [4.2000e+03, 4.2010e+03, 4.2020e+03,  ..., 4.7970e+03,
          4.7980e+03, 4.7990e+03]],

        [[4.8000e+03, 4.8010e+03, 4.8020e+03,  ..., 5.3970e+03,
          5.3980e+03, 5.3990e+03],
         [5.4000e+03, 5.4010e+03, 5.4020e+03,  ..., 5.9970e+03,
          5.9980e+03, 5.9990e+03]]])

In [74]:
import time
st = time.perf_counter()
pooling_1(A)
et = time.perf_counter()
print("pooling 1", et-st)

st = time.perf_counter()
pooling_2(A)
et = time.perf_counter()
print("pooling 2", et-st)

pooling 1 5.510290400000031
pooling 2 0.0008459000000584638


In [75]:
B  = A.permute((0, 2, 1))

In [76]:
A

tensor([[[0.0000e+00, 1.0000e+00, 2.0000e+00,  ..., 5.9700e+02,
          5.9800e+02, 5.9900e+02],
         [6.0000e+02, 6.0100e+02, 6.0200e+02,  ..., 1.1970e+03,
          1.1980e+03, 1.1990e+03]],

        [[1.2000e+03, 1.2010e+03, 1.2020e+03,  ..., 1.7970e+03,
          1.7980e+03, 1.7990e+03],
         [1.8000e+03, 1.8010e+03, 1.8020e+03,  ..., 2.3970e+03,
          2.3980e+03, 2.3990e+03]],

        [[2.4000e+03, 2.4010e+03, 2.4020e+03,  ..., 2.9970e+03,
          2.9980e+03, 2.9990e+03],
         [3.0000e+03, 3.0010e+03, 3.0020e+03,  ..., 3.5970e+03,
          3.5980e+03, 3.5990e+03]],

        [[3.6000e+03, 3.6010e+03, 3.6020e+03,  ..., 4.1970e+03,
          4.1980e+03, 4.1990e+03],
         [4.2000e+03, 4.2010e+03, 4.2020e+03,  ..., 4.7970e+03,
          4.7980e+03, 4.7990e+03]],

        [[4.8000e+03, 4.8010e+03, 4.8020e+03,  ..., 5.3970e+03,
          5.3980e+03, 5.3990e+03],
         [5.4000e+03, 5.4010e+03, 5.4020e+03,  ..., 5.9970e+03,
          5.9980e+03, 5.9990e+03]]])